In [1]:
from diffusers import StableDiffusionInpaintPipeline
import torch
from PIL import Image


pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "/mnt/c/Users/11042/.cache/huggingface/hub/models--wyyadd--sd-1.5-inpainting/snapshots/fcac56836a29b84e1d03a5b08f2550890a510baa",
    revision="fp16",
    torch_dtype=torch.float16,
    safety_checker=None
).to("cuda")


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

An error occurred while trying to fetch /mnt/c/Users/11042/.cache/huggingface/hub/models--wyyadd--sd-1.5-inpainting/snapshots/fcac56836a29b84e1d03a5b08f2550890a510baa/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /mnt/c/Users/11042/.cache/huggingface/hub/models--wyyadd--sd-1.5-inpainting/snapshots/fcac56836a29b84e1d03a5b08f2550890a510baa/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
/home/nervld/miniconda3/envs/omini/lib/python3.10/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
An error occurred while trying to fetch /mnt/c/Users/11042/.cache/huggingface/hub/models--wyyadd--sd-1.5-inpainting/snapshots/fcac56836a29b84e1d03a5b08f2550890a510baa/unet: Error no file named diffusion_pytorch_model.safetensors found in director

In [2]:
from peft import PeftModel, PeftConfig
# 加载lora
lora_path = "/home/nervld/gitclone/OminiControl/test/pytorch_lora_weights.safetensors"  # 替换为你的 LoRA 模型路径
pipe.load_lora_weights(lora_path)

In [5]:
prompt = ""
image = Image.open("./datasets/inpainting/vton_hd/garment_person_pair_new/00024_00.jpg")
mask_image = Image.open("./datasets/inpainting/vton_hd/model_mask_pair/00024_00.jpg")
#image and mask_image should be PIL images.
#The mask structure is white for inpainting and black for keeping as is
image = pipe(prompt=prompt, image=image, mask_image=mask_image).images[0]
image.save("./t_output.png")

  0%|          | 0/50 [00:00<?, ?it/s]

In [37]:
# 把数据集里面的所有图片都进行inpainting然后取出
# /home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/garment_person_pair_new里面是image，
# /home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/model_mask_pair里面是mask
# 把image和mask对应起来，进行inpainting
# 把inpainting的结果保存到/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/garment_person_pair_new_inpainting里面

# 获取文件列表
import os
from tqdm import tqdm

image_list = os.listdir("/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/garment_person_pair_new")

# 将result_list按照名称排序
image_list = sorted(image_list)

mask_list = os.listdir("/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/model_mask_pair")
pipe.set_progress_bar_config(disable=True)

total_count = 100
count = 0
# 把image和mask对应起来，进行inpainting
for image_file in tqdm(image_list):
    if count >= total_count:
        break
    count += 1
    image_path = os.path.join("/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/garment_person_pair_new", image_file)
    mask_path = os.path.join("/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/model_mask_pair", image_file)
    image = Image.open(image_path)
    mask = Image.open(mask_path)
    with torch.no_grad():
        inpainting_result = pipe(prompt="",image=image, mask_image=mask,num_inference_steps=20,guidance_scale=1.0,generator=torch.Generator(device=pipe.device).manual_seed(42)).images[0]
    # 把图片切分为左右两张图，右边的图是正确结果
    width, height = inpainting_result.size
    right_image = inpainting_result.crop((width // 2, 0, width, height))
    right_image.save(os.path.join("/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/new_result", image_file))


  0%|          | 0/11647 [00:00<?, ?it/s]

  1%|          | 100/11647 [02:52<5:31:39,  1.72s/it]


In [42]:
# 将garment_person_pair_new_inpainting里面的图片切分为左右两张图，右边的图是正确结果
real_image_list = os.listdir("/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/garment_person_pair_new")
real_image_list = sorted(real_image_list)

count = 0
total_count = 100

for real_image_file in real_image_list:
    if count >= total_count:
        break
    count += 1
    real_image_path = os.path.join("/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/garment_person_pair_new", real_image_file)
    real_image = Image.open(real_image_path)
    width, height = real_image.size
    real_image = real_image.crop((width // 2, 0, width, height))
    real_image = real_image.resize((256, 512))
    real_image.save(os.path.join("/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/real_image", real_image_file))


In [52]:
# 分别使用SSIM, FID, LPIPS评估inpainting结果
# 使用SSIM评估inpainting结果
import numpy as np
from skimage.metrics import structural_similarity as ssim

# 计算SSIM
def calculate_ssim(image1, image2):
    image1 = np.array(image1)
    image2 = np.array(image2)
    return ssim(image1, image2, win_size=3)

# 使用LPIPS评估inpainting结果
import lpips
import lpips
loss_fn_alex = lpips.LPIPS(net='alex') # best forward scores
# loss_fn_vgg = lpips.LPIPS(net='vgg') # closer to "traditional" perceptual loss, when used for optimization

def calculate_lpips(image1, image2):
    image1 = np.array(image1)
    image2 = np.array(image2)
    return loss_fn_alex(image1, image2)

result_path = "/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/new_result"
result_list = os.listdir(result_path)


# 去/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/garment_person_pair_new里面找到与result_list对应的图片

# 计算SSIM, FID, LPIPS
ssim_list = []
lpips_list = []

for result_file in result_list:
    result_path = os.path.join(result_path, result_file)
    result_image = Image.open(result_path)
    correct_image = Image.open(os.path.join("/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/real_image", result_file))
 
    ssim_list.append(calculate_ssim(result_image, correct_image))
    lpips_list.append(calculate_lpips(result_image, correct_image))
    break

print(ssim_list)
print(lpips_list)



Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/home/nervld/miniconda3/envs/omini/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/nervld/miniconda3/envs/omini/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth


/home/nervld/miniconda3/envs/omini/lib/python3.10/site-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_path,

TypeError: unsupported operand type(s) for -: 'numpy.ndarray' and 'Tensor'

In [51]:
import torch
import torchvision
import torchvision.transforms as transforms
from pytorch_fid import fid_score

# 准备真实数据分布和生成模型的图像数据
real_images_folder = '/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/real_image'
generated_images_folder = '/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/new_result'

# 加载预训练的Inception-v3模型
inception_model = torchvision.models.inception_v3(pretrained=True)

# 计算FID距离值
fid_value = fid_score.calculate_fid_given_paths([real_images_folder, generated_images_folder],batch_size=100,device="cuda",dims=2048,num_workers=1)
print('FID value:', fid_value)



100%|██████████| 1/1 [00:00<00:00,  1.37it/s]


FID value: -5.061391325966724e-05


### 加上网络结构的改造尝试

In [2]:
# 从/home/nervld/gitclone/OminiControl/test/condition_image_encoder/pytorch_lora_weights.safetensors加载一个condition_image_encoder
from transformers import CLIPVisionModelWithProjection
image_encoder_id = "/mnt/c/Users/11042/.cache/huggingface/hub/models--openai--clip-vit-large-patch14/snapshots/32bd64288804d66eefd0ccbe215aa642df71cc41"
condition_image_encoder = CLIPVisionModelWithProjection.from_pretrained(image_encoder_id).to(device="cuda")
condition_image_encoder.eval()
print("√")

√


In [7]:
import torch
from transformers import CLIPVisionModelWithProjection
from safetensors.torch import load_file

import torch
import json
from transformers import CLIPVisionModelWithProjection
from peft import PeftModel, LoraConfig
from safetensors.torch import load_file  # 用于加载 safetensors 格式的权重


# 构造一个lora_config
lora_config = LoraConfig(
    init_lora_weights="gaussian",
    lora_alpha=128,
    r=128,
    target_modules=["q_proj", "v_proj", "k_proj", "out_proj","in_proj","fc","proj","ln_1","ln_2","ln_3","ln_post","ln_pre","ln_post_attn","ln_pre_attn"],
    bias="none"
)

# 3. 创建 PeftModel，并绑定 LoRA 适配器（**不使用 load_adapter**）
lora_model = PeftModel(condition_image_encoder, lora_config)



# 2. 加载本地的 safetensors 格式的 LoRA 权重
lora_path = "/home/nervld/gitclone/OminiControl/test/condition_image_encoder/condition_image_encoder.pth"  # 替换为你的 LoRA 文件路径
adapter_weights = torch.load(lora_path)

print("模型的keys",lora_model.state_dict().keys())
print("需要加载的keys",adapter_weights.keys())

# 3. 定义一个函数来将 LoRA 权重应用到模型上
def apply_lora_to_model(model, adapter_weights):
    model_state_dict = model.state_dict()
    for adapter_key in adapter_weights.keys():
        # 将 adapter_key 转换为模型中的对应键
        if "vision_model" in adapter_key:
            model_key = adapter_key.replace("vision_model", "base_model.model.vision_model").replace("weight", "default.weight")
            
        else:
            continue  # 如果不是 vision_model 的权重，则跳过

        # 检查模型中是否存在对应的键
        if model_key in model_state_dict:
            # 获取 LoRA 权重
            lora_weight = adapter_weights[adapter_key]
            # 获取模型中的权重
            model_weight = model_state_dict[model_key]

            # 检查权重维度是否匹配
            if lora_weight.shape == model_weight.shape:
                # 将 LoRA 权重加到模型权重上
                model_weight += lora_weight.to(model_weight.device)
            else:
                print(f"Warning: Shape mismatch for {model_key}. Skipping.")
        else:
            print(f"Warning: Key {model_key} not found in model state dict. Skipping.")

# 4. 应用 LoRA 权重
apply_lora_to_model(lora_model, adapter_weights)

# 5. 模型转移到设备上（例如 GPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lora_model.to(device)

# 6. 示例：使用模型进行推理
pixel_values = torch.randn(1, 3, 224, 224).to(device)  # 示例输入
outputs = lora_model(pixel_values)
print(outputs.image_embeds.shape)  # 预期输出 torch.Size([1, 512])

模型的keys odict_keys(['base_model.model.vision_model.embeddings.class_embedding', 'base_model.model.vision_model.embeddings.patch_embedding.weight', 'base_model.model.vision_model.embeddings.position_embedding.weight', 'base_model.model.vision_model.pre_layrnorm.weight', 'base_model.model.vision_model.pre_layrnorm.bias', 'base_model.model.vision_model.encoder.layers.0.self_attn.k_proj.base_layer.weight', 'base_model.model.vision_model.encoder.layers.0.self_attn.k_proj.base_layer.bias', 'base_model.model.vision_model.encoder.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.vision_model.encoder.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.vision_model.encoder.layers.0.self_attn.v_proj.base_layer.weight', 'base_model.model.vision_model.encoder.layers.0.self_attn.v_proj.base_layer.bias', 'base_model.model.vision_model.encoder.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.vision_model.encoder.layers.0.self_attn.v_proj.lora_B.defa

/tmp/ipykernel_200786/599331741.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapter_weights = torch.load(lora_path)


In [8]:
# # 加载lora
# lora_path = "/home/nervld/gitclone/OminiControl/test/condition_image_encoder"  # 替换为你的 LoRA 模型路径


# # 加载本地lora
# lora_path = "/home/nervld/gitclone/OminiControl/test/condition_image_encoder/pytorch_lora_weights.safetensors"  # 替换为你的 LoRA 模型路径
# condition_image_encoder.load_adapter(lora_path, adapter_name="condition_image_encoder2")

# 从/home/nervld/gitclone/OminiControl/test/projection/projection.pth 加载一个projection
projection = torch.nn.Sequential(
        torch.nn.Linear(1024, 1024),
        torch.nn.ReLU(),
        torch.nn.Linear(1024, 768)
    ).to(device="cuda", dtype=pipe.dtype)
projection.load_state_dict(torch.load("/home/nervld/gitclone/OminiControl/test/projection/projection.pth"))


/tmp/ipykernel_200786/882617722.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  projection.load_state_dict(torch.load("/home/nervld/gitclone/OminiControl/test/projectio

<All keys matched successfully>

In [15]:
number = 0
# 1. 加载测试图像和mask
inpaint_init = Image.open(f"/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/garment_person_pair_new/00{number:03d}_00.jpg")
mask_image = Image.open(f"/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/model_mask_pair/00{number:03d}_00.jpg")
condition_image = Image.open(f"/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/condition_garment/00{number:03d}_00.png")


# 2. 调整大小并确保格式正确
height = width = 512
inpaint_init = inpaint_init.resize((width, height))
mask_image = mask_image.resize((width, height))
condition_image = condition_image.resize((224, 224))  # CLIP需要224x224
condition_image = condition_image.convert("RGB")

with torch.no_grad():
    init_image = pipe.image_processor.preprocess(
        inpaint_init,
        height=height,
        width=width
    ).to(device=pipe.device, dtype=pipe.dtype)
    
    # 4. 预处理mask
    from PIL import Image
    import numpy as np
    mask = torch.from_numpy(np.array(mask_image.convert("L"))[None, None, ...])
    mask = mask.to(device=pipe.device, dtype=pipe.dtype) / 255.0

    # 5. 创建masked image
    masked_image = init_image * (1 - mask)
    # 移入cuda
    masked_image = masked_image.to(device=pipe.device, dtype=pipe.dtype)
    
    # 6. 预处理condition image
    condition_tensor = pipe.image_processor.preprocess(
        condition_image,
        height=224,
        width=224
    ).to(device=pipe.device, dtype=pipe.dtype)
    
    # 7. 获取condition embeddings
    condition_image_embeds = lora_model(condition_tensor)
    condition_image_embeds = condition_image_embeds.last_hidden_state.to(dtype=pipe.dtype)
    condition_image_embeds = projection(condition_image_embeds)

    # 移入cuda
    condition_image_embeds = condition_image_embeds.to(device=pipe.device, dtype=pipe.dtype)
    print(condition_image_embeds.shape)
    # 拼接一个相同维度的全0张量
    # condition_image_embeds = torch.cat([condition_image_embeds, torch.zeros_like(condition_image_embeds)], dim=1)
    print(condition_image_embeds.shape)
    # 8. 准备latents
    num_inference_steps = 20
    pipe.scheduler.set_timesteps(num_inference_steps)
    
    image = pipe(
        image=masked_image,
        mask_image=mask,
        num_inference_steps=num_inference_steps,
        generator=torch.Generator(device=pipe.device).manual_seed(42),
        # prompt = "",
        prompt_embeds=condition_image_embeds, # 将condition image的embeds作为prompt_embeds传入
        direct_use_prompt_embeds=True, # 直接使用prompt_embeds
        # 这里修改了__call__函数，用direct_use_prompt_embeds=True来直接使用prompt_embeds
        guidance_scale=1.0, # 设置guidance_scale为1.0，是为了禁用classifier-free guidance
    ).images[0]

# 10. 保存结果
image.save("./t_output1.png")

torch.Size([1, 257, 768])
torch.Size([1, 257, 768])


  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
from src.train.model import InpaintingModel2

def get_config(config_path):
    import os
    import yaml
    assert config_path is not None, "Please set the XFL_CONFIG environment variable"
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    return config
config = get_config("/home/nervld/gitclone/OminiControl/train/config/inpainting_512_new.yaml")
training_config = config["train"]


pl_module = InpaintingModel2(
    inpainting_pipe_id=config["inpainting_model_path"],
    image_encoder_id=config["image_encoder_path"],
    exsist_unet_lora_path=config.get("exsist_unet_lora_path", None),
    exsist_image_encoder_lora_path=config.get("exsist_image_encoder_lora_path", None),
    exsist_projection_path=config.get("exsist_projection_path", None),
    lora_config=training_config["lora_config"],
    device=f"cuda",
    dtype=getattr(torch, config["dtype"]),
    optimizer_config=training_config["optimizer"],
    model_config=config.get("model", {}),
    gradient_checkpointing=training_config.get("gradient_checkpointing", False),
)

In [8]:
# 1. 加载测试图像和mask
import os
import yaml
import torch
from PIL import Image
import numpy as np

number = 19

inpaint_init = Image.open(f"/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/garment_person_pair_new/00{number:03d}_00.jpg")
mask_image = Image.open(f"/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/model_mask_pair/00{number:03d}_00.jpg")
condition_image = Image.open(f"/home/nervld/gitclone/OminiControl/datasets/inpainting/vton_hd/condition_garment/00{number:03d}_00.png")

# 2. 调整大小并确保格式正确
height = width = 512
inpaint_init = inpaint_init.resize((width, height))
mask_image = mask_image.resize((width, height))
condition_image = condition_image.resize((224, 224))  # CLIP需要224x224
condition_image = condition_image.convert("RGB")

with torch.no_grad():
    # 3. 预处理输入
    init_image = pl_module.inpainting_pipe.image_processor.preprocess(
        inpaint_init,
        height=height,
        width=width
    ).to(device=pl_module.device, dtype=pl_module.dtype)
    
    # 4. 预处理mask
    mask = torch.from_numpy(np.array(mask_image.convert("L"))[None, None, ...])
    mask = mask.to(device=pl_module.device, dtype=pl_module.dtype) / 255.0
    
    # 5. 创建masked image
    masked_image = init_image * (1 - mask)
    # 移入cuda
    masked_image = masked_image.to(device=pl_module.device, dtype=pl_module.dtype)
    
    # 6. 预处理condition image
    condition_tensor = pl_module.inpainting_pipe.image_processor.preprocess(
        condition_image,
        height=224,
        width=224
    ).to(device=pl_module.device, dtype=pl_module.dtype)
    
    # 7. 获取condition embeddings
    condition_image_embeds = pl_module.condition_image_encoder(condition_tensor)
    condition_image_embeds = condition_image_embeds.last_hidden_state
    condition_image_embeds = pl_module.projection(condition_image_embeds)

    # 移入cuda
    condition_image_embeds = condition_image_embeds.to(device=pl_module.device, dtype=pl_module.dtype)
    
    # 8. 准备latents
    num_inference_steps = 20
    pl_module.inpainting_pipe.scheduler.set_timesteps(num_inference_steps)
    
    # 9. 生成图像
    image = pl_module.inpainting_pipe(
        image=masked_image,
        mask_image=mask,
        num_inference_steps=num_inference_steps,
        generator=torch.Generator(device=pl_module.device),
        prompt_embeds=condition_image_embeds, # 将condition image的embeds作为prompt_embeds传入
        direct_use_prompt_embeds=True, # 直接使用prompt_embeds
        # 这里修改了__call__函数，用direct_use_prompt_embeds=True来直接使用prompt_embeds
        guidance_scale=1.0 # 设置guidance_scale为1.0，是为了禁用classifier-free guidance
    ).images[0]

image.save("./t_output1.png")

  0%|          | 0/20 [00:00<?, ?it/s]

In [67]:
prompt = "the person in the right is wearing clothing exactly matching the style shown in the left reference image"
image = Image.open("./datasets/inpainting/vton_hd/garment_person_pair_new/00203_00.jpg")
mask_image = Image.open("./datasets/inpainting/vton_hd/model_mask_pair/00203_00.jpg")
#image and mask_image should be PIL images.
#The mask structure is white for inpainting and black for keeping as is
image = pipe(prompt=prompt, image=image, mask_image=mask_image).images[0]
image.save("./t_output.png")

  0%|          | 0/50 [00:00<?, ?it/s]

In [68]:
prompt = ""
image = Image.open("./datasets/inpainting/vton_hd/garment_person_pair_new/00203_00.jpg")
mask_image = Image.open("./datasets/inpainting/vton_hd/model_mask_pair/00203_00.jpg")
#image and mask_image should be PIL images.
#The mask structure is white for inpainting and black for keeping as is
image = pipe(prompt=prompt, image=image, mask_image=mask_image).images[0]
image.save("./t_output.png")

  0%|          | 0/50 [00:00<?, ?it/s]